## Import

In [7]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## News sentiment

In [10]:
GDELT_folder_path = '../data/processed/'
GDELT_file1 = os.path.join(GDELT_folder_path, "GDELT_week_weibull_04Apr_update.csv")

# Read CSV files
df = pd.read_csv(GDELT_file1)

# Select only specific columns
df = df.reset_index(drop=True).rename(columns={'Unnamed: 0': 'DATE'})
# Convert DATE column to datetime format
df['DATE'] = pd.to_datetime(df['DATE'])

# Sort by date to ensure correct time series plotting
df = df.sort_values(by='DATE')

In [11]:
GDELT_df = df.rename(columns={'smoothed_sentiment_weibull_week': 'news_smoothed_sentiment_weibull_week'})
GDELT_df['DATE'] = pd.to_datetime(GDELT_df['DATE']).dt.tz_localize('UTC')
GDELT_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11021 entries, 0 to 11020
Data columns (total 2 columns):
 #   Column                                Non-Null Count  Dtype              
---  ------                                --------------  -----              
 0   DATE                                  11021 non-null  datetime64[ns, UTC]
 1   news_smoothed_sentiment_weibull_week  11021 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 172.3 KB


In [12]:
GDELT_df

,DATE,news_smoothed_sentiment_weibull_week
0,2024-01-01 00:00:00+00:00,0.000000
1,2024-01-01 01:00:00+00:00,-0.110741
2,2024-01-01 02:00:00+00:00,2.833173
3,2024-01-01 03:00:00+00:00,10.543272
4,2024-01-01 04:00:00+00:00,21.441960
...,...,...
11016,2025-04-04 00:00:00+00:00,-1751.699125
11017,2025-04-04 01:00:00+00:00,-1752.509797
11018,2025-04-04 02:00:00+00:00,-1750.695053
11019,2025-04-04 03:00:00+00:00,-1747.355304


# BTC pq check

In [35]:
import pandas as pd

# Read the Parquet file
processed_data_folder_path = '../data/processed/'
btc_reddit_data_path = os.path.join(processed_data_folder_path, "reddit_relevant_posts_till_april_sentiment.parquet")
df = pd.read_parquet(btc_reddit_data_path)

# Show basic info
# print(df.info())


In [36]:
df

,post_id,title,body,author,upvotes,downvotes,score,num_comments,url,created_at,subreddit,date_created,is_relevant,year,week,day,top_comments,overall_sentiment,sentiment_score
0,1jjyomf,Help recovering my Samourai Wallet?,A long time ago I had got myself some BTC in a...,haileliamen,2,0,2,22,https://www.reddit.com/r/BitcoinBeginners/comm...,2025-03-26 00:11:46+00:00,BitcoinBeginners,2025-03-26,Yes.,2025,13,2,"[https://freedom.tech/samourai-to-sparrow/, Sc...",neutral,0
1,1jjyskb,Look what I can buy with my bitcoin gift card,No text body (link post),eagle_eye_johnson,0,0,0,9,https://v.redd.it/jzzzrr26jvqe1,2025-03-26 00:16:57+00:00,btc,2025-03-26,Yes,2025,13,2,"[[removed], This really is the epitome of flog...",negative,-1
2,1jk0di6,Bitcoin is Still Misunderstood,No text body (link post),InclineDumbbellPress,1237,0,1237,193,https://i.redd.it/xllts9e2txqe1.png,2025-03-26 01:33:07+00:00,CryptoCurrency,2025-03-26,Yes,2025,13,2,"[If you don't understand it, it's legit not to...",neutral,0
3,1jk0qkj,First US Engineered Hydro-Cooled Bitcoin Miner...,No text body (link post),btcxio,5,0,5,3,https://news.bitcoinprotocol.org/first-us-engi...,2025-03-26 01:51:18+00:00,BitcoinMining,2025-03-26,Yes,2025,13,2,[I've talked to the Auradine team. Good people...,neutral,0
4,1jk33yu,You can remove money with only a send address?,"I may be crazy, but when I try to send Bitcoin...",Lonely_cricket143,36,0,36,23,https://www.reddit.com/r/BitcoinBeginners/comm...,2025-03-26 03:56:00+00:00,BitcoinBeginners,2025-03-26,Yes,2025,13,2,[That doesn't sound right. You are saying the...,neutral,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,1jpyypr,Jack Mallers explains Bitcoin in 30 mins,One of the best and most enthusiastic explanat...,Bubbly_Ice3836,191,0,191,10,https://x.com/TFTC21/status/1894820980297519565,2025-04-02 20:29:39+00:00,Bitcoin,2025-04-02,Yes,2025,14,2,[YouTube mirror https://youtu.be/d7ID3fKAFQM?s...,neutral,0
354,1jq1g1h,Bear Market,Trump just signed the tarrifs on middle east c...,CurrentAd8334,16,0,16,63,https://www.reddit.com/r/CryptoMarkets/comment...,2025-04-02 22:11:43+00:00,CryptoMarkets,2025-04-02,Yes.,2025,14,2,[Whales and institutions were buying BTC. Smar...,neutral,0
355,1jq1gbt,Algorand currently has $410 million worth of s...,No text body (link post),gigabyteIO,154,0,154,22,https://app.rwa.xyz/stocks,2025-04-02 22:12:02+00:00,CryptoCurrency,2025-04-02,Yes,2025,14,2,"[Algo is the tits. Time to tak'em prime time,...",neutral,0
356,1jq2gq5,"They’re Shaking You Out—Stay Strong, Don’t Sell",As Tom Petty said—it’s free fallin’. Hold your...,AncientKey1976,77,0,77,147,https://www.reddit.com/r/CryptoMarkets/comment...,2025-04-02 22:56:11+00:00,CryptoMarkets,2025-04-02,Yes,2025,14,2,"[U can’t shake the DEAD, Can you guys look at ...",neutral,0


## BTC Price and Reddit Sentiment

In [13]:
processed_data_folder_path = '../data/processed/'
btc_reddit_data_path = os.path.join(processed_data_folder_path, "btc_usdt_hourly_processed_with_sentiment_till_4_april.csv")
btc_reddit_data = pd.read_csv(btc_reddit_data_path)

# Convert DATE column to datetime format
btc_reddit_data['datetime'] = pd.to_datetime(btc_reddit_data['datetime'])
btc_reddit_data = btc_reddit_data.sort_values(by='datetime')

In [14]:
btc_reddit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10207 entries, 0 to 10206
Data columns (total 47 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   datetime                         10207 non-null  datetime64[ns, UTC]
 1   OPEN                             10207 non-null  float64            
 2   HIGH                             10207 non-null  float64            
 3   LOW                              10207 non-null  float64            
 4   CLOSE                            10207 non-null  float64            
 5   TOTAL_TRADES                     10207 non-null  int64              
 6   TOTAL_TRADES_BUY                 10207 non-null  int64              
 7   TOTAL_TRADES_SELL                10207 non-null  int64              
 8   VOLUME                           10207 non-null  float64            
 9   VOLUME_BUY                       10207 non-null  float64            
 10

In [15]:
btc_reddit_data

,datetime,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,VOLUME,VOLUME_BUY,VOLUME_SELL,return,log_return,hourly_volatility,EMA_8,EMA_13,EMA_21,EMA_Signal,EMA_short,EMA_long,MACD,Signal_Line,MACD_Signal,MACD_Hist,SMA20,STD20,Upper_BB,Lower_BB,RSI_14,BB_RSI_Signal,RSI_Signal,SMA_50,SMA_200,GoldenCross_Signal,%K,%D,Stochastic_Signal,OBV,OBV_EMA20,OBV_Signal,H-L,H-PC,L-PC,TR,ATR,smoothed_sentiment_weibull_week,post_density
0,2024-01-27 13:00:00+00:00,41750.00,41806.55,41725.21,41742.44,35569,10171,25398,419.38468,187.70449,231.68019,-0.000181,-0.000181,0.002460,41729.550796,41714.525206,41617.761701,1,41720.541302,41533.453446,187.087856,256.308536,0,-69.220681,41814.7665,145.139023,42105.044546,41524.488454,45.171588,0,0,40928.6316,40765.60695,0,72.793587,71.453152,0,-419.38468,-65.712408,0,81.34,56.55,24.79,81.34,175.710000,0.000000,2
1,2024-01-27 14:00:00+00:00,41742.44,41818.39,41731.01,41766.00,27238,12281,14957,493.43458,233.95644,259.47814,0.000564,0.000564,0.002463,41737.650619,41721.878748,41631.237910,1,41727.534948,41550.679117,176.855831,240.417995,0,-63.562164,41797.8160,128.274352,42054.364704,41541.267296,48.792247,0,0,40966.7554,40767.06695,0,79.088844,75.565353,0,493.43458,-12.460314,1,87.38,75.95,11.43,87.38,140.500000,0.027001,0
2,2024-01-27 15:00:00+00:00,41766.00,41894.87,41733.60,41814.00,32410,14004,18406,581.25856,268.18654,313.07202,0.001149,0.001149,0.002486,41754.617148,41735.038927,41647.852645,1,41740.837263,41570.184367,170.652896,226.464975,0,-55.812079,41785.2915,112.071762,42009.435023,41561.147977,49.710914,0,0,41004.7586,40769.71570,0,91.914496,81.265642,0,581.25856,44.084341,0,161.27,128.87,32.40,161.27,109.318333,0.164191,0
3,2024-01-27 16:00:00+00:00,41814.00,41895.74,41791.22,41810.18,30278,13664,16614,609.18361,315.81592,293.36769,-0.000091,-0.000091,0.002476,41766.964449,41745.773366,41662.609678,1,41751.505377,41587.961822,163.543555,213.880691,0,-50.337136,41776.9415,102.865601,41982.672702,41571.210298,48.277493,0,0,41040.7034,40772.31665,0,90.893788,87.299043,0,-609.18361,-18.131655,-1,104.52,81.74,22.78,104.52,101.921667,0.300527,0
4,2024-01-27 17:00:00+00:00,41810.18,41845.34,41774.07,41779.28,24681,9330,15351,448.23397,165.47870,282.75527,-0.000739,-0.000739,0.002416,41769.701238,41750.560028,41673.216071,1,41755.778396,41602.133538,153.644857,201.833525,0,-48.188667,41765.9665,88.683186,41943.332872,41588.600128,47.852739,0,0,41080.2042,40774.22120,0,82.637275,88.481853,-1,-448.23397,-59.093780,0,71.27,35.16,36.11,71.27,99.801667,0.435303,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10202,2025-03-27 15:00:00+00:00,87297.33,87575.00,86752.58,86974.07,180420,91975,88445,1554.66361,699.72490,854.93871,-0.003703,-0.003710,0.003902,87030.204495,87088.309186,87142.490052,-1,87079.095604,87166.116350,-87.020747,-52.194077,0,-34.826670,87201.6400,316.582835,87834.805670,86568.474330,43.373971,0,0,87378.1368,85785.53265,0,45.884868,38.966585,0,-1554.66361,-216.223678,-1,822.42,277.67,544.75,822.42,736.608333,-3.084545,1
10203,2025-03-27 16:00:00+00:00,86974.07,87271.43,86805.20,86835.24,129979,66364,63615,671.69986,244.03675,427.66311,-0.001596,-0.001597,0.003900,86986.879052,87052.156445,87114.558229,-1,87041.579357,87141.606991,-100.027634,-61.760788,0,-38.266846,87179.4020,326.261700,87831.925399,86526.878601,37.651530,0,0,87354.3936,85802.64380,0,35.092315,50.664024,0,-671.69986,-259.602362,0,466.23,297.36,168.87,466.23,732.878333,-3.131449,0
10204,2025-03-27 17:00:00+00:00,86835.24,87150.95,86796.00,86913.05,120897,50367,70530,596.49192,257.41179,339.08013,0.000896,0.000896,0.003883,86970.472596,87032.284096,87096.239299,-1,87021.805610,87124.676844,-102.871234,-69.982878,0,-32.888356,87169.5720,331.395193,87832.362386,86506.781614,37.296263,0,0,87337.1672,85819.57220,0,46.513386,42.496856,0,596.49192,-178.069573,1,354.

In [18]:
# Create a new dataframe with selected features
btc_reddit_data_selected_features = btc_reddit_data[['datetime',  
                       'EMA_8','EMA_13','EMA_21','EMA_Signal','EMA_short','EMA_long', 
                       'MACD','Signal_Line','MACD_Hist', # MACD Signal
                       'Upper_BB', 'Lower_BB', # Bollinger Bands Signal
                       'RSI_14', 'BB_RSI_Signal', 'RSI_Signal', # RSI Signal
                       'SMA_50', 'SMA_200', 'GoldenCross_Signal', # Golden Cross Signal
                       '%K', '%D', 'Stochastic_Signal', # Stochastic Oscillator Signal
                       'OBV','OBV_EMA20', 'OBV_Signal', # OBV Signal
                       'ATR', # ATR, volatility
                       'smoothed_sentiment_weibull_week',
                       'return',
                       'hourly_volatility',
                       'TOTAL_TRADES', # Trade is number of trades (someone put buy or sell order)
                       'VOLUME' # Volume is amount of cryptocurrency traded in BTC
                      ]]


In [19]:
btc_reddit_data_selected_features.rename(columns={'smoothed_sentiment_weibull_week': 'reddit_smoothed_sentiment_weibull_week'}, inplace=True)

C:\Users\Justin\AppData\Local\Temp\ipykernel_15352\4068817824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_reddit_data_selected_features.rename(columns={'smoothed_sentiment_weibull_week': 'reddit_smoothed_sentiment_weibull_week'}, inplace=True)


## Telegram Data

In [21]:
processed_data_folder_path = '../data/processed/'
telegram_data_path = os.path.join(processed_data_folder_path, "tele_sentiment_04Apr.csv")
btc_telegram_data = pd.read_csv(telegram_data_path, index_col=0).reset_index().rename(columns = {'index': 'date'})

# Convert DATE column to datetime format
btc_telegram_data['date'] = pd.to_datetime(btc_telegram_data['date'])
btc_telegram_data = btc_telegram_data.sort_values(by='date').reset_index(drop = True)

In [23]:
tele_data = btc_telegram_data[['date', 'smoothed_sentiment_weibull_week']]
tele_data.rename(columns={'smoothed_sentiment_weibull_week':'tele_smoothed_sentiment_weibull_week'},inplace=True)
tele_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11009 entries, 0 to 11008
Data columns (total 2 columns):
 #   Column                                Non-Null Count  Dtype              
---  ------                                --------------  -----              
 0   date                                  11009 non-null  datetime64[ns, UTC]
 1   tele_smoothed_sentiment_weibull_week  11009 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 172.1 KB


C:\Users\Justin\AppData\Local\Temp\ipykernel_15352\2275741366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tele_data.rename(columns={'smoothed_sentiment_weibull_week':'tele_smoothed_sentiment_weibull_week'},inplace=True)


In [24]:
tele_data

,date,tele_smoothed_sentiment_weibull_week
0,2024-01-01 06:00:00+00:00,0.000000
1,2024-01-01 07:00:00+00:00,0.002271
2,2024-01-01 08:00:00+00:00,0.070902
3,2024-01-01 09:00:00+00:00,0.139164
4,2024-01-01 10:00:00+00:00,0.206704
...,...,...
11004,2025-04-03 18:00:00+00:00,4.696564
11005,2025-04-03 19:00:00+00:00,4.648396
11006,2025-04-03 20:00:00+00:00,4.596983
11007,2025-04-03 21:00:00+00:00,4.542238


## MERGE

In [25]:
merged_df = btc_reddit_data_selected_features.merge(GDELT_df, left_on='datetime', right_on = "DATE", how='left').merge(tele_data, left_on='datetime', right_on = "date", how='left')
merged_df.drop(columns=['date','DATE'], inplace=True)

In [28]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10207 entries, 0 to 10206
Data columns (total 32 columns):
 #   Column                                  Non-Null Count  Dtype              
---  ------                                  --------------  -----              
 0   datetime                                10207 non-null  datetime64[ns, UTC]
 1   EMA_8                                   10207 non-null  float64            
 2   EMA_13                                  10207 non-null  float64            
 3   EMA_21                                  10207 non-null  float64            
 4   EMA_Signal                              10207 non-null  int64              
 5   EMA_short                               10207 non-null  float64            
 6   EMA_long                                10207 non-null  float64            
 7   MACD                                    10207 non-null  float64            
 8   Signal_Line                             10207 non-null  float64            


In [ ]:
merged_df_path = '../data/processed/merged_df_04_April.csv'

merged_df.to_csv(merged_df_path, index=False)